<a href="https://colab.research.google.com/github/MoonJaeHoon/Dacon_Behavioral_Data_Analysis/blob/master/Ch0_%EC%A0%84%EC%B2%98%EB%A6%AC_reduce_memory_of_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
import os
print( '변경 전 기본 경로 :' )
print(  os.getcwd() )
os.chdir('./drive/My Drive/데이콘_게임데이터분석')
print( '변경 후 파일 경로 :' )
print(  os.getcwd()  )

import numpy as np
import pandas as pd

from tqdm import tqdm_notebook              # 진행바
from sklearn.metrics import roc_auc_score   # AUC 스코어 계산
from sklearn.model_selection import KFold   # K-fold CV    
#from bayes_opt import BayesianOptimization  # 베이지안 최적화 라이브러리  
from functools import partial               # 함수 변수 고정
import lightgbm as lgb                      # LightGBM 라이브러리
import warnings                             
warnings.filterwarnings("ignore")           # 경고 문구 미표시
%time
train = pd.read_csv('./train.csv')

#%time
#test = pd.read_pickle('./test.pkl')

변경 전 기본 경로 :
/content
변경 후 파일 경로 :
/content/drive/My Drive/데이콘_게임데이터분석
CPU times: user 2 µs, sys: 1 µs, total: 3 µs
Wall time: 5.48 µs


In [0]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 67091776 entries, 0 to 67091775
Data columns (total 7 columns):
 #   Column          Dtype  
---  ------          -----  
 0   game_id         int64  
 1   winner          int64  
 2   time            float64
 3   player          int64  
 4   species         object 
 5   event           object 
 6   event_contents  object 
dtypes: float64(1), int64(3), object(3)
memory usage: 3.5+ GB


In [6]:
def reduce_mem_usage(props):
    start_mem_usg = props.memory_usage().sum() / 1024**2 
    print("Memory usage of properties dataframe is :",start_mem_usg," MB")
    NAlist = [] # Keeps track of columns that have missing values filled in. 
    for col in props.columns:
        if props[col].dtype != object:  # Exclude strings
            
            # Print current column type
            #print("******************************")
            #print("Column: ",col)
            #print("dtype before: ",props[col].dtype)
            
            # make variables for Int, max and min
            IsInt = False
            mx = props[col].max()
            mn = props[col].min()
            
            # Integer does not support NA, therefore, NA needs to be filled
            if not np.isfinite(props[col]).all(): 
                NAlist.append(col)
                props[col].fillna(mn-1,inplace=True)  
                   
            # test if column can be converted to an integer
            asint = props[col].fillna(0).astype(np.int64)
            result = (props[col] - asint)
            result = result.sum()
            if result > 0.01:
                IsInt = True

            
            # Make Integer/unsigned Integer datatypes
            if IsInt:
                if mn >= 0:
                    if mx < 255:
                        props[col] = props[col].astype(np.uint8)
                    elif mx < 65535:
                        props[col] = props[col].astype(np.uint16)
                    elif mx < 4294967295:
                        props[col] = props[col].astype(np.uint32)
                    else:
                        props[col] = props[col].astype(np.uint64)
                else:
                    if mn > np.iinfo(np.int8).max:
                        props[col] = props[col].astype(np.int8)
                    elif mn > np.iinfo(np.int16).max:
                        props[col] = props[col].astype(np.int16)
                    elif mn > np.iinfo(np.int32).max:
                        props[col] = props[col].astype(np.int32)
                    elif mn > np.iinfo(np.int64).max:
                        props[col] = props[col].astype(np.int64)    
            
            # Make float datatypes 32 bit
            else:
                props[col] = props[col].astype(np.float32)
            
            # Print new column type
            #print("dtype after: ",props[col].dtype)
            #print("******************************")
    
    # Print final result
    print("___MEMORY USAGE AFTER COMPLETION:___")
    mem_usg = props.memory_usage().sum() / 1024**2 
    print("Memory usage is: ",mem_usg," MB")
    print("This is ",100*mem_usg/start_mem_usg,"% of the initial size")
    return props, NAlist
    
train, NAlist = reduce_mem_usage(train)
print("_________________")
print("Warning: the following columns have missing values filled with 'df['column_name'].min() -1': ")
print("_________________")
print("")
print(NAlist)

Memory usage of properties dataframe is : 3583.0875244140625  MB
___MEMORY USAGE AFTER COMPLETION:___
Memory usage is:  2367.3971557617188  MB
This is  66.07142972732312 % of the initial size
_________________
_________________

[]


In [8]:
train.to_pickle('train.pkl')
train = pd.read_pickle('train.pkl')
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 67091776 entries, 0 to 67091775
Data columns (total 7 columns):
 #   Column          Dtype  
---  ------          -----  
 0   game_id         float32
 1   winner          float32
 2   time            uint8  
 3   player          float32
 4   species         object 
 5   event           object 
 6   event_contents  object 
dtypes: float32(3), object(3), uint8(1)
memory usage: 2.3+ GB


In [14]:
del train
test = pd.read_csv('test.csv')
print(test.info())
test, NAlist = reduce_mem_usage(test)
print("-"*30)
print("Warning: the following columns have missing values filled with 'df['column_name'].min() -1': ")
print("-"*30)
print("")
print(NAlist)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28714849 entries, 0 to 28714848
Data columns (total 6 columns):
 #   Column          Dtype  
---  ------          -----  
 0   game_id         int64  
 1   time            float64
 2   player          int64  
 3   species         object 
 4   event           object 
 5   event_contents  object 
dtypes: float64(1), int64(2), object(3)
memory usage: 1.3+ GB
None
Memory usage of properties dataframe is : 1314.4615936279297  MB
___MEMORY USAGE AFTER COMPLETION:___
Memory usage is:  903.6923837661743  MB
This is  68.75000290209869 % of the initial size
------------------------------
------------------------------

[]


In [15]:
test.to_pickle('test.pkl')
test = pd.read_pickle('test.pkl')
print(test.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28714849 entries, 0 to 28714848
Data columns (total 6 columns):
 #   Column          Dtype  
---  ------          -----  
 0   game_id         float32
 1   time            uint8  
 2   player          float32
 3   species         object 
 4   event           object 
 5   event_contents  object 
dtypes: float32(2), object(3), uint8(1)
memory usage: 903.7+ MB
None
